In [3]:
%run '~/git/Bibliotecas.ipynb'

In [4]:
r_pa=pd.read_csv('~/git/Importacao-por-SH-e-Municipio/temp/r_pa.csv')
r_pa.head(1)

,Ano1,Mês,Ano2,Ano3,Ano4,Ano5,Ano6,Ano7,Ano8,Ano9,Ano10,Ano11,Ano12,KT_f,miFOB_f,KT_c,miFOB_c,mm
0,2007,1,2006,2006,2006,2006,2006,2006,2006,2006,2006,2006,2006,7.75,1.577625,4.9404,11.409323,440.032479


In [5]:
ano_tipo=['Ano1','Ano4','Ano7','Ano10']

In [6]:
quant=[ 'KT_f', 'miFOB_f', 'KT_c', 'miFOB_c','mm']

In [7]:
m_input=['rel_preço_ad','mm_ad','miFOB_f_ad', 'KT_c_ad', 'miFOB_c_ad']

In [8]:
import itertools as it

In [9]:
%%time
q=pd.DataFrame()

for a_t in ano_tipo:
    
    r_pa_at=r_pa[[a_t,'Mês']+quant].dropna()
    a_c=r_pa_at[[a_t,'Mês']].drop_duplicates()
    a_c=a_c.groupby(a_t)['Mês'].count().reset_index()
    a_c=a_c[a_c['Mês']==12][[a_t]]
    r_pa_at=r_pa_at.merge(a_c)
    r_pa_at=r_pa_at.groupby(a_t)[quant].sum().reset_index()
    r_pa_at['rel_preço']=(r_pa_at.miFOB_f/r_pa_at.KT_f)/(r_pa_at.miFOB_c/r_pa_at.KT_c)
    ad1=r_pa_at[[a_t,'KT_f']]
    ad2=r_pa_at.drop(columns=['KT_f'])
    ad2[a_t]=ad2[a_t]+1
    ad2=ad2.rename(columns={'miFOB_f':'miFOB_f_ad', 'KT_c':'KT_c_ad', 'miFOB_c':'miFOB_c_ad','rel_preço':'rel_preço_ad','mm':'mm_ad'})
    r_pa_ad_at=ad1.merge(ad2)

    
    a_i_a=r_pa_ad_at[a_t].min()
    
    anos_fim_treino=r_pa_ad_at[a_t].unique().tolist()
    
    for a_f_t in anos_fim_treino:
        
        for duração_treino in range(5,15,2):
            
            a_i_t=a_f_t-duração_treino
            
            if a_i_t>=a_i_a:

                r_pa_ad_at_train=r_pa_ad_at[
                    (r_pa_ad_at[a_t]<=a_f_t)
                    &
                    (r_pa_ad_at[a_t]>a_i_t)]

                train=r_pa_ad_at_train
                

                # Cria arquivos vazios para receber tests
                test_f=pd.DataFrame()
                #test_p=pd.DataFrame()

                # Se o ano para previsão f estiver contido em r_pa_ad_t
                if r_pa_ad_at_train[a_t].max()+1<=r_pa_ad_at[a_t].max():

                    # Define o test_f
                    test_f=r_pa_ad_at[r_pa_ad_at[a_t]==r_pa_ad_at_train[a_t].max()+1]


                if len(test_f)>0:


                    for n_comb in range(1,len(m_input)+1):

                        iterator=it.combinations(m_input,n_comb)

                        c_l=list(next(iterator))

                        while len(c_l)>0:

                            train_t=train[['KT_f']+c_l].dropna()

                            X_train=train_t.drop(columns='KT_f')
                            y_train=train_t.KT_f

                            LR.fit(X_train,y_train)
                            RFR.fit(X_train,y_train)

                            if len(test_f)>0:
                                test_f_t=test_f[['KT_f']+c_l]
                                X_test_f=test_f_t.drop(columns='KT_f')
                                y_test_f=test_f_t.KT_f.values[0]
                                test_f_LR=LR.predict(X_test_f)[0]
                                test_f_RFR=RFR.predict(X_test_f)[0]
                                
                            else:
                                y_test_f=np.nan
                                test_f_prev=np.nan


                            loop_result=pd.DataFrame({
                                'ano_tipo':[a_t],
                                'ano_ini_treino':[a_i_t],
                                'ano_fim_treino':[a_f_t],
                                'duração_treino':[duração_treino],
                                'c_l': ','.join(c_l),
                                'y_test_f':[y_test_f],
                                'test_f_LR':[test_f_LR],
                                'test_f_RFR':[test_f_RFR]
                                

                            })

                            q=pd.concat([q,loop_result])



                            try: c_l=list(next(iterator))
                            except: c_l=[]



display(q)

,ano_tipo,ano_ini_treino,ano_fim_treino,duração_treino,c_l,y_test_f,test_f_LR,test_f_RFR
0,Ano1,2008,2013,5,rel_preço_ad,71.228923,78.674632,98.332447
0,Ano1,2008,2013,5,mm_ad,71.228923,77.037154,56.518282
0,Ano1,2008,2013,5,miFOB_f_ad,71.228923,87.953915,82.161232
0,Ano1,2008,2013,5,KT_c_ad,71.228923,-38.883329,50.506497
0,Ano1,2008,2013,5,miFOB_c_ad,71.228923,77.677716,81.821664
...,...,...,...,...,...,...,...,...
0,Ano10,2008,2021,13,"rel_preço_ad,mm_ad,miFOB_f_ad,miFOB_c_ad",95.704447,99.945028,102.340616
0,Ano10,2008,2021,13,"rel_preço_ad,mm_ad,KT_c_ad,miFOB_c_ad",95.704447,-79.823155,110.006555
0,Ano10,2008,2021,13,"rel_preço_ad,miFOB_f_ad,KT_c_ad,miFOB_c_ad",95.704447,-82.963353,111.329698
0,Ano10,2008,2021,13,"mm_ad,miFOB_f_ad,KT_c_ad,miFOB_c_ad",95.704447,72.694794,107.167982


CPU times: user 6min 25s, sys: 1.81 s, total: 6min 27s
Wall time: 6min 27s


In [10]:
q['test_f%_LR'] =round(100*(q['test_f_LR' ]-q['y_test_f'])/q['y_test_f'])
q['test_f%_RFR']=round(100*(q['test_f_RFR']-q['y_test_f'])/q['y_test_f'])

In [11]:
q.to_csv('~/git/Importacao-por-SH-e-Municipio/temp/q1.csv',index=False)